In [1]:
import os

os.chdir(r"/home/yousinator/AuspicesAI/ScytheEx/ai")

from internal_lib import DataSetInformation, MachineLearning
import pandas as pd

In [2]:
titles = [
    "dur",
    "proto",
    "service",
    "state",
    "stcpb",
    "dtcpb",
    "synack",
    "ackdat",
    "trans_depth",
    "response_body_len",
    "ct_src_dport_ltm",
    "ct_dst_sport_ltm",
    "is_ftp_login",
    "ct_ftp_cmd",
    "ct_flw_http_mthd",
    "is_sm_ips_ports",
    "attack_cat",
]


train = pd.read_csv("data/training.csv")
test = pd.read_csv("data/testing.csv")

combined_df = pd.concat([train, test], ignore_index=True)

# Shuffle the combined DataFrame
df = combined_df.sample(frac=1).reset_index(drop=True)

df = df[titles]

df_info = DataSetInformation(df)

df.drop_duplicates(inplace=True)

In [15]:
services = df["service"].unique()



array(['dns', '-', 'http', 'ssh', 'smtp', 'ftp-data', 'ftp', 'pop3',
       'ssl', 'dhcp', 'snmp', 'irc', 'radius'], dtype=object)

In [19]:
df["state"].unique()

array(['CON', 'INT', 'ECO', 'FIN', 'RST', 'REQ', 'no', 'ACC', 'CLO',
       'URN', 'PAR'], dtype=object)

In [10]:
import redis
import json

def fetch_data_from_db():
    # Setup Redis connection
    r = redis.Redis(host="5.tcp.eu.ngrok.io", port=15140, db=0)

    # Subscribe to the channel
    pubsub = r.pubsub()
    pubsub.subscribe("processed_data_channel")

    print("Subscribed to 'processed_data_channel'. Waiting for data...")
    # Listen for new messages
    for message in pubsub.listen():
        if message["type"] == "message":
            data = json.loads(message["data"])
            print("Received data:", data)
            df = pd.DataFrame([data])
            return df

input = fetch_data_from_db()

Subscribed to 'processed_data_channel'. Waiting for data...
Received data: {'dur': '0', 'proto': 'TCP', 'service': 'HTTPS', 'state': '', 'stcpb': '808560832', 'dtcpb': '3086082638', 'synack': '0', 'ackdat': '0', 'trans_depth': '0', 'response_body_len': '0', 'ct_src_dport_ltm': '0', 'ct_dst_sport_ltm': '0', 'is_ftp_login': '0', 'ct_ftp_cmd': '0', 'ct_flw_http_mthd': '0', 'is_sm_ips_ports': '0'}


In [13]:
input_df = input.copy()
input_df = input_df.astype(
    {
        "dur": "float64",
        "synack": "float64",
        "ackdat": "float64",
        "stcpb": "int64",
        "dtcpb": "int64",
        "trans_depth": "int64",
        "response_body_len": "int64",
        "ct_src_dport_ltm": "int64",
        "ct_dst_sport_ltm": "int64",
        "is_ftp_login": "int64",
        "ct_ftp_cmd": "int64",
        "ct_flw_http_mthd": "int64",
        "is_sm_ips_ports": "int64",
    }
)

input_df[input_df.select_dtypes(include="object").columns] = input_df[
    input_df.select_dtypes(include="object").columns
].apply(lambda x: x.str.lower())

input_df

,dur,proto,service,state,stcpb,dtcpb,synack,ackdat,trans_depth,response_body_len,ct_src_dport_ltm,ct_dst_sport_ltm,is_ftp_login,ct_ftp_cmd,ct_flw_http_mthd,is_sm_ips_ports
0,0.0,tcp,https,,808560832,3086082638,0.0,0.0,0,0,0,0,0,0,0,0


In [16]:
df.shape[0]

129859

In [17]:
df["status"].unique()

KeyError: 'status'